In [1]:
!pip install --upgrade tensorflow typing

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached tensorflow-2.9.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (511.7 MB)
  Using cached typing-3.7.4.3-py3-none-any.whl
  Using cached numpy-1.22.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Using cached tensorflow_estimator-2.9.0-py2.py3-none-any.whl (438 kB)
  Using cached tensorboard-2.9.0-py3-none-any.whl (5.8 MB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached keras-2.9.0-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Un

In [116]:
###
#
#  Declare a tf model with signatures
#  Inspired by https://github.com/erdememekligil/oop-tensorflow-serving
#
###

import abc
from typing import Type
import tensorflow as tf

class ImageClassifierModel(tf.keras.Model):
    """
    A base class for image classification models. Subclasses must implement create_model_io function that returns the
    input and the output of the model.
    SavedModel of this class will have two serving signatures. The default one (serving_default) calculates predictions
    using images as 4d arrays. The other signature, serving_bytes, operates on base64 encoded image bytes.
    """

    def __init__(self, class_names: [str], input_shape=(3, 150, 150), *args, **kwargs):
        self.class_names = tf.constant(class_names)
        inp, out = self.create_model_io(input_shape)
        kwargs = {**kwargs, **{'inputs': inp, 'outputs': out}}
        super().__init__(*args, **kwargs)

    @abc.abstractmethod
    def create_model_io(self, input_shape: tuple = (3, 150, 150)):
        pass

    def call(self, inputs, training=None, mask=None):
        return super(ImageClassifierModel, self).call(inputs, training=training, mask=mask)

    def get_config(self):
        return super(ImageClassifierModel, self).get_config()
    
    
    @tf.function(input_signature=[tf.TensorSpec(shape=None, dtype=tf.string)])
    def predict_bytes_image(self, image_buffer):
        """
        Predict using encoded image bytes.
        :param image: png, jpeg, bmp, gif encoded image bytes.
        :return: prediction result.
        """
        
        image = tf.image.decode_jpeg(image_buffer, channels=3)
        image = tf.image.resize(image, (150, 150))
        image = tf.image.convert_image_dtype(image, dtype=tf.uint8)
        image = tf.reshape(image, (1, 3, 150, 150))
        preprocessed = tf.keras.applications.vgg16.preprocess_input(image)
        
        model_output = self.call(preprocessed)
        model_output['class_names'] = self.class_names
    
        return model_output
    
    def save(self, filepath, overwrite=True, include_optimizer=True, save_format=None, signatures=None, 
             options=None, save_traces=True):
        """
        Saves model with custom signatures.
        serving_default: predict using 4 array image (numpy/tensor).
        serving_bytes: predict using base64 encoded image bytes.
        """
        if signatures is None:
            signatures = dict()
        signatures['serving_default'] = self.predict_bytes_image
        super().save(f'{filepath}/1', overwrite, include_optimizer, save_format, signatures, options, save_traces)
        

class VGG16Model(ImageClassifierModel):
    """
    This model fine-tunes a pretrained VGG16.
    No need to rescale the data, pre-processing is done by the model.
    """

    def create_model_io(self, input_shape: tuple = (3, 150, 150)):
        """
        Loads a pre-trained VGG16.
        :return: the input and the output.
        """
        inp = tf.keras.layers.Input(input_shape, dtype=tf.uint8)
        
        embeddings = tf.keras.applications.vgg16.VGG16(
            weights='imagenet', include_top=False, input_shape=input_shape)(inp)
        
        flatten_layer = tf.keras.layers.Flatten()(embeddings)
        
        dense_layer_1 = tf.keras.layers.Dense(50, activation='relu')(flatten_layer)
        
        dense_layer_2 = tf.keras.layers.Dense(20, activation='relu')(dense_layer_1)
        
        logits = tf.keras.layers.Dense(len(self.class_names), activation=None, name='logits')(dense_layer_2)
        
        prediction = tf.keras.layers.Activation('softmax')(logits)
        
        out = {
            'embeddings': tf.reshape(embeddings, [1, -1]),
            'logits': logits,
            'prediction': prediction
        }

        return inp, out
    

In [117]:
###
#
#  Do the training and save the model ...
#
###

my_model = VGG16Model(class_names=['class_1', 'class_2', 'class_3'])

# Train ...

my_model.save('./best_model')

INFO:tensorflow:Assets written to: ./best_model/1/assets


INFO:tensorflow:Assets written to: ./best_model/1/assets


In [118]:
###
#
#  Mock spin up the TF serving server ...
#
###

loaded = tf.saved_model.load('./best_model/1')
infer = loaded.signatures["serving_default"]

In [102]:
!pip install dioptra

Found existing installation: dioptra 0.2.7
Uninstalling dioptra-0.2.7:
  Successfully uninstalled dioptra-0.2.7
running install
/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/setuptools/command/easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
writing dioptra.egg-info/PKG-INFO
writing dependency_links to dioptra.egg-info/dependency_links.txt
writing requirements to dioptra.egg-info/requires.txt
writing top-level names to dioptra.egg-info/top_level.txt
reading manifest file 'dioptra.egg-info/SOURCES.txt'
adding license file 'LICENSE.md'
writing manifest file 'dioptra.egg-info/SOURCES.txt'

In [119]:
###
#
#  Get inference
#
###

import io

file = tf.keras.utils.get_file(
    "grace_hopper.jpg",
    "https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg")
image = tf.keras.utils.load_img(file, target_size=[150, 150])

img_byte_arr = io.BytesIO()
image.save(img_byte_arr, format='JPEG')
img_byte_arr = img_byte_arr.getvalue()
outputs = infer(tf.constant(img_byte_arr))

In [114]:
###
#
#  Send to Dioptra
#
###

from dioptra.api import Logger
from dioptra.supported_types import SupportedTypes
import uuid

model_id = 'my_vgg16_classifier'
model_version = 'v1.1'

api_key = 'my_api_key'

dioptra_logger = Logger(api_key=api_key)

datapoint = {
    'request_id': str(uuid.uuid4()),
    'model_id': model_id,
    'model_version': model_version,
    'model_type': SupportedTypes.TEXT_CLASSIFIER,
    'prediction': {
        'class_name': outputs['class_names'].numpy().tolist()[0],
        'prediction': outputs['prediction'].numpy().tolist()[0],
        'logits': outputs['logits'].numpy().tolist()[0]
    },
    'embeddings': outputs['embeddings'].numpy().tolist()[0]
}

dioptra_logger.commit(datapoint)